## Analysis 1: Best Case Hours

In [4]:
# Critical Path Analysis 

# Problem description from Williams (2013, pages 94-98)
# Williams, H. Paul. 2013. Model Building in Mathematical Programming (fifth edition). New York: Wiley. [ISBN-13: 978-1-118-44333-0]

# Python PuLP solution prepared by Thomas W. Miller
# Revised April 20, 2023
# Implemented using activities dictionary with derived start_times and end_times
# rather than time decision variables as in Williams (2013)

#from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize

from pulp import *

# Create a dictionary of the activities and their durations
##FIRST LINE IS BEST CASE
activities = {'A_DescribeProduct':2, 'B_Marketing':4, 'C_Brochure':8, 'D1_Requirements':8, 'D2_SoftwareDesign':8, 'D3_SystemDesign':8, 'D4_Coding':160, 'D5_Documentation':16, 'D6_UnitTesting':20, 'D7_SystemTesting':20, 'D8_Package':12, 'E_Survey':20, 'F_Pricing':10, 'G_Implementation': 40, 'H_Proposal': 40}

##FOLLOWING LINE IS EXPECTED CASE
#activities = {'A_DescribeProduct':4, 'B_Marketing':8, 'C_Brochure':16, 'D1_Requirements':16, 'D2_SoftwareDesign':16, 'D3_SystemDesign':16, 'D4_Coding':320, 'D5_Documentation':32, 'D6_UnitTesting':40, 'D7_SystemTesting':40, 'D8_Package':24, 'E_Survey':40, 'F_Pricing':20, 'G_Implementation': 80, 'H_Proposal': 80}

##FOLLOWNG LINE IS WORST CASE
#activities = {'A_DescribeProduct':6, 'B_Marketing':12, 'C_Brochure':24, 'D1_Requirements':24, 'D2_SoftwareDesign':24, 'D3_SystemDesign':24, 'D4_Coding':480, 'D5_Documentation':48, 'D6_UnitTesting':60, 'D7_SystemTesting':60, 'D8_Package':36, 'E_Survey':60, 'F_Pricing':30, 'G_Implementation': 120, 'H_Proposal': 160}


# Create a list of the activities
activities_list = list(activities.keys())

# Create a dictionary of the activity precedences
precedences = {'A_DescribeProduct':[], 'B_Marketing':[], 'C_Brochure':['A_DescribeProduct'], 'D1_Requirements':['A_DescribeProduct'], 'D2_SoftwareDesign':['D1_Requirements'], 'D3_SystemDesign':['D1_Requirements'], 'D4_Coding':['D2_SoftwareDesign', 'D3_SystemDesign'], 'D5_Documentation':['D4_Coding'], 'D6_UnitTesting':['D4_Coding'], 'D7_SystemTesting':['D6_UnitTesting'], 'D8_Package':['D5_Documentation', 'D7_SystemTesting'], 'E_Survey':['B_Marketing', 'C_Brochure'], 'F_Pricing':['D8_Package', 'E_Survey'], 'G_Implementation': ['A_DescribeProduct', 'D8_Package'], 'H_Proposal': ['F_Pricing', 'G_Implementation']}


# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} days in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)



Critical Path time:
A_DescribeProduct starts at time 0
B_Marketing starts at time 0
H_Proposal ends at 310.0 days in duration

Solution variable values:
end_A_DescribeProduct = 2.0
end_B_Marketing = 4.0
end_C_Brochure = 10.0
end_D1_Requirements = 10.0
end_D2_SoftwareDesign = 18.0
end_D3_SystemDesign = 18.0
end_D4_Coding = 178.0
end_D5_Documentation = 194.0
end_D6_UnitTesting = 198.0
end_D7_SystemTesting = 218.0
end_D8_Package = 230.0
end_E_Survey = 30.0
end_F_Pricing = 240.0
end_G_Implementation = 270.0
end_H_Proposal = 310.0
start_A_DescribeProduct = 0.0
start_B_Marketing = 0.0
start_C_Brochure = 2.0
start_D1_Requirements = 2.0
start_D2_SoftwareDesign = 10.0
start_D3_SystemDesign = 10.0
start_D4_Coding = 18.0
start_D5_Documentation = 178.0
start_D6_UnitTesting = 178.0
start_D7_SystemTesting = 198.0
start_D8_Package = 218.0
start_E_Survey = 10.0
start_F_Pricing = 230.0
start_G_Implementation = 230.0
start_H_Proposal = 270.0


In [5]:
print(prob)

Critical_Path:
MINIMIZE
1*end_A_DescribeProduct + 1*end_B_Marketing + 1*end_C_Brochure + 1*end_D1_Requirements + 1*end_D2_SoftwareDesign + 1*end_D3_SystemDesign + 1*end_D4_Coding + 1*end_D5_Documentation + 1*end_D6_UnitTesting + 1*end_D7_SystemTesting + 1*end_D8_Package + 1*end_E_Survey + 1*end_F_Pricing + 1*end_G_Implementation + 1*end_H_Proposal + 0
SUBJECT TO
A_DescribeProduct_duration: end_A_DescribeProduct - start_A_DescribeProduct
 = 2

B_Marketing_duration: end_B_Marketing - start_B_Marketing = 4

C_Brochure_duration: end_C_Brochure - start_C_Brochure = 8

C_Brochure_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_C_Brochure >= 0

D1_Requirements_duration: end_D1_Requirements - start_D1_Requirements = 8

D1_Requirements_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_D1_Requirements >= 0

D2_SoftwareDesign_duration: end_D2_SoftwareDesign - start_D2_SoftwareDesign
 = 8

D2_SoftwareDesign_predecessor_D1_Requirements: - end_D1_Requirements
 + start_D

## Expected Case Hours

In [6]:
# Critical Path Analysis 

# Problem description from Williams (2013, pages 94-98)
# Williams, H. Paul. 2013. Model Building in Mathematical Programming (fifth edition). New York: Wiley. [ISBN-13: 978-1-118-44333-0]

# Python PuLP solution prepared by Thomas W. Miller
# Revised April 20, 2023
# Implemented using activities dictionary with derived start_times and end_times
# rather than time decision variables as in Williams (2013)

#from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize

from pulp import *

# Create a dictionary of the activities and their durations
##FIRST LINE IS BEST CASE
#activities = {'A_DescribeProduct':2, 'B_Marketing':4, 'C_Brochure':8, 'D1_Requirements':8, 'D2_SoftwareDesign':8, 'D3_SystemDesign':8, 'D4_Coding':160, 'D5_Documentation':16, 'D6_UnitTesting':20, 'D7_SystemTesting':20, 'D8_Package':12, 'E_Survey':20, 'F_Pricing':10, 'G_Implementation': 40, 'H_Proposal': 40}

##FOLLOWING LINE IS EXPECTED CASE
activities = {'A_DescribeProduct':4, 'B_Marketing':8, 'C_Brochure':16, 'D1_Requirements':16, 'D2_SoftwareDesign':16, 'D3_SystemDesign':16, 'D4_Coding':320, 'D5_Documentation':32, 'D6_UnitTesting':40, 'D7_SystemTesting':40, 'D8_Package':24, 'E_Survey':40, 'F_Pricing':20, 'G_Implementation': 80, 'H_Proposal': 80}

##FOLLOWNG LINE IS WORST CASE
#activities = {'A_DescribeProduct':6, 'B_Marketing':12, 'C_Brochure':24, 'D1_Requirements':24, 'D2_SoftwareDesign':24, 'D3_SystemDesign':24, 'D4_Coding':480, 'D5_Documentation':48, 'D6_UnitTesting':60, 'D7_SystemTesting':60, 'D8_Package':36, 'E_Survey':60, 'F_Pricing':30, 'G_Implementation': 120, 'H_Proposal': 160}


# Create a list of the activities
activities_list = list(activities.keys())

# Create a dictionary of the activity precedences
precedences = {'A_DescribeProduct':[], 'B_Marketing':[], 'C_Brochure':['A_DescribeProduct'], 'D1_Requirements':['A_DescribeProduct'], 'D2_SoftwareDesign':['D1_Requirements'], 'D3_SystemDesign':['D1_Requirements'], 'D4_Coding':['D2_SoftwareDesign', 'D3_SystemDesign'], 'D5_Documentation':['D4_Coding'], 'D6_UnitTesting':['D4_Coding'], 'D7_SystemTesting':['D6_UnitTesting'], 'D8_Package':['D5_Documentation', 'D7_SystemTesting'], 'E_Survey':['B_Marketing', 'C_Brochure'], 'F_Pricing':['D8_Package', 'E_Survey'], 'G_Implementation': ['A_DescribeProduct', 'D8_Package'], 'H_Proposal': ['F_Pricing', 'G_Implementation']}


# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} days in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)



Critical Path time:
A_DescribeProduct starts at time 0
B_Marketing starts at time 0
H_Proposal ends at 620.0 days in duration

Solution variable values:
end_A_DescribeProduct = 4.0
end_B_Marketing = 8.0
end_C_Brochure = 20.0
end_D1_Requirements = 20.0
end_D2_SoftwareDesign = 36.0
end_D3_SystemDesign = 36.0
end_D4_Coding = 356.0
end_D5_Documentation = 388.0
end_D6_UnitTesting = 396.0
end_D7_SystemTesting = 436.0
end_D8_Package = 460.0
end_E_Survey = 60.0
end_F_Pricing = 480.0
end_G_Implementation = 540.0
end_H_Proposal = 620.0
start_A_DescribeProduct = 0.0
start_B_Marketing = 0.0
start_C_Brochure = 4.0
start_D1_Requirements = 4.0
start_D2_SoftwareDesign = 20.0
start_D3_SystemDesign = 20.0
start_D4_Coding = 36.0
start_D5_Documentation = 356.0
start_D6_UnitTesting = 356.0
start_D7_SystemTesting = 396.0
start_D8_Package = 436.0
start_E_Survey = 20.0
start_F_Pricing = 460.0
start_G_Implementation = 460.0
start_H_Proposal = 540.0


In [7]:
print(prob)

Critical_Path:
MINIMIZE
1*end_A_DescribeProduct + 1*end_B_Marketing + 1*end_C_Brochure + 1*end_D1_Requirements + 1*end_D2_SoftwareDesign + 1*end_D3_SystemDesign + 1*end_D4_Coding + 1*end_D5_Documentation + 1*end_D6_UnitTesting + 1*end_D7_SystemTesting + 1*end_D8_Package + 1*end_E_Survey + 1*end_F_Pricing + 1*end_G_Implementation + 1*end_H_Proposal + 0
SUBJECT TO
A_DescribeProduct_duration: end_A_DescribeProduct - start_A_DescribeProduct
 = 4

B_Marketing_duration: end_B_Marketing - start_B_Marketing = 8

C_Brochure_duration: end_C_Brochure - start_C_Brochure = 16

C_Brochure_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_C_Brochure >= 0

D1_Requirements_duration: end_D1_Requirements - start_D1_Requirements = 16

D1_Requirements_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_D1_Requirements >= 0

D2_SoftwareDesign_duration: end_D2_SoftwareDesign - start_D2_SoftwareDesign
 = 16

D2_SoftwareDesign_predecessor_D1_Requirements: - end_D1_Requirements
 + star

## Worst Case Hours

In [8]:
# Critical Path Analysis 

# Problem description from Williams (2013, pages 94-98)
# Williams, H. Paul. 2013. Model Building in Mathematical Programming (fifth edition). New York: Wiley. [ISBN-13: 978-1-118-44333-0]

# Python PuLP solution prepared by Thomas W. Miller
# Revised April 20, 2023
# Implemented using activities dictionary with derived start_times and end_times
# rather than time decision variables as in Williams (2013)

#from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize

from pulp import *

# Create a dictionary of the activities and their durations
##FIRST LINE IS BEST CASE
#activities = {'A_DescribeProduct':2, 'B_Marketing':4, 'C_Brochure':8, 'D1_Requirements':8, 'D2_SoftwareDesign':8, 'D3_SystemDesign':8, 'D4_Coding':160, 'D5_Documentation':16, 'D6_UnitTesting':20, 'D7_SystemTesting':20, 'D8_Package':12, 'E_Survey':20, 'F_Pricing':10, 'G_Implementation': 40, 'H_Proposal': 40}

##FOLLOWING LINE IS EXPECTED CASE
#activities = {'A_DescribeProduct':4, 'B_Marketing':8, 'C_Brochure':16, 'D1_Requirements':16, 'D2_SoftwareDesign':16, 'D3_SystemDesign':16, 'D4_Coding':320, 'D5_Documentation':32, 'D6_UnitTesting':40, 'D7_SystemTesting':40, 'D8_Package':24, 'E_Survey':40, 'F_Pricing':20, 'G_Implementation': 80, 'H_Proposal': 80}

##FOLLOWNG LINE IS WORST CASE
activities = {'A_DescribeProduct':6, 'B_Marketing':12, 'C_Brochure':24, 'D1_Requirements':24, 'D2_SoftwareDesign':24, 'D3_SystemDesign':24, 'D4_Coding':480, 'D5_Documentation':48, 'D6_UnitTesting':60, 'D7_SystemTesting':60, 'D8_Package':36, 'E_Survey':60, 'F_Pricing':30, 'G_Implementation': 120, 'H_Proposal': 160}


# Create a list of the activities
activities_list = list(activities.keys())

# Create a dictionary of the activity precedences
precedences = {'A_DescribeProduct':[], 'B_Marketing':[], 'C_Brochure':['A_DescribeProduct'], 'D1_Requirements':['A_DescribeProduct'], 'D2_SoftwareDesign':['D1_Requirements'], 'D3_SystemDesign':['D1_Requirements'], 'D4_Coding':['D2_SoftwareDesign', 'D3_SystemDesign'], 'D5_Documentation':['D4_Coding'], 'D6_UnitTesting':['D4_Coding'], 'D7_SystemTesting':['D6_UnitTesting'], 'D8_Package':['D5_Documentation', 'D7_SystemTesting'], 'E_Survey':['B_Marketing', 'C_Brochure'], 'F_Pricing':['D8_Package', 'E_Survey'], 'G_Implementation': ['A_DescribeProduct', 'D8_Package'], 'H_Proposal': ['F_Pricing', 'G_Implementation']}


# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)

# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}

# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"

    
# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} days in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)



Critical Path time:
A_DescribeProduct starts at time 0
B_Marketing starts at time 0
H_Proposal ends at 970.0 days in duration

Solution variable values:
end_A_DescribeProduct = 6.0
end_B_Marketing = 12.0
end_C_Brochure = 30.0
end_D1_Requirements = 30.0
end_D2_SoftwareDesign = 54.0
end_D3_SystemDesign = 54.0
end_D4_Coding = 534.0
end_D5_Documentation = 582.0
end_D6_UnitTesting = 594.0
end_D7_SystemTesting = 654.0
end_D8_Package = 690.0
end_E_Survey = 90.0
end_F_Pricing = 720.0
end_G_Implementation = 810.0
end_H_Proposal = 970.0
start_A_DescribeProduct = 0.0
start_B_Marketing = 0.0
start_C_Brochure = 6.0
start_D1_Requirements = 6.0
start_D2_SoftwareDesign = 30.0
start_D3_SystemDesign = 30.0
start_D4_Coding = 54.0
start_D5_Documentation = 534.0
start_D6_UnitTesting = 534.0
start_D7_SystemTesting = 594.0
start_D8_Package = 654.0
start_E_Survey = 30.0
start_F_Pricing = 690.0
start_G_Implementation = 690.0
start_H_Proposal = 810.0


In [9]:
print(prob)

Critical_Path:
MINIMIZE
1*end_A_DescribeProduct + 1*end_B_Marketing + 1*end_C_Brochure + 1*end_D1_Requirements + 1*end_D2_SoftwareDesign + 1*end_D3_SystemDesign + 1*end_D4_Coding + 1*end_D5_Documentation + 1*end_D6_UnitTesting + 1*end_D7_SystemTesting + 1*end_D8_Package + 1*end_E_Survey + 1*end_F_Pricing + 1*end_G_Implementation + 1*end_H_Proposal + 0
SUBJECT TO
A_DescribeProduct_duration: end_A_DescribeProduct - start_A_DescribeProduct
 = 6

B_Marketing_duration: end_B_Marketing - start_B_Marketing = 12

C_Brochure_duration: end_C_Brochure - start_C_Brochure = 24

C_Brochure_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_C_Brochure >= 0

D1_Requirements_duration: end_D1_Requirements - start_D1_Requirements = 24

D1_Requirements_predecessor_A_DescribeProduct: - end_A_DescribeProduct
 + start_D1_Requirements >= 0

D2_SoftwareDesign_duration: end_D2_SoftwareDesign - start_D2_SoftwareDesign
 = 24

D2_SoftwareDesign_predecessor_D1_Requirements: - end_D1_Requirements
 + sta